# Function Calling Rest API with local model using mistral-inference

Function calling allows Mistral models to connect to external tools. By integrating Mistral models with external tools such as user defined functions or APIs, users can easily build applications catering to specific use cases and practical problems. In this guide, for instance, we wrote two functions for tracking a Pet Store's Pets and User info. We can use these two tools to provide answers for pet-related queries.

At a glance, there are four steps with function calling:

- User: specify tools and query
- Model: Generate function arguments if applicable
- User: Execute function to obtain tool results
- Model: Generate final answer

In this guide, we will walk through a simple example to demonstrate how function calling works with Mistral models in these four steps.

Before we get started, let’s assume we have an OpenAPI spec end-points consisting of Pet store information. When users ask questions about this API, they can use certain tools to answer questions about this data. This is just an example to emulate an external database via API that the LLM cannot directly access.

In [ ]:
!pip install pandas mistral-inference

Setup functions to make REST API call. We take example of pet store from [Swagger Editor](https://editor.swagger.io/)  
We download the openapi.json specification.

Example curl query to get information of a Pet by PetID

`
curl -X 'GET' \
  'https://petstore3.swagger.io/api/v3/pet/1' \
  -H 'accept: application/json'
`

Example curl query to get information of a User by username
`
curl -X 'GET' \
  'https://petstore3.swagger.io/api/v3/user/user1' \
  -H 'accept: application/json'
`  

# Function Calling for REST API

## Step 1. User: specify tools and query

### Tools

Users can define all the necessary tools for their use cases.

- In many cases, we might have multiple tools at our disposal. For example, let’s consider we have two functions as our two tools: `retrieve_pet_info` and `retreive_user_info` to retrieve pet and user info given `petID` and `username`.

In [1]:
import requests
def retrieve_pet_info(petId: int) -> str:
    try:
        method = 'GET'
        headers=None
        data=None
        url =  'https://petstore3.swagger.io/api/v3/pet/' + str(petId)
        response = requests.request(method, url, headers=headers, data=data)
        # Raise an exception if the response was unsuccessful
        response.raise_for_status()
        #response = make_api_call('GET', url + str(petId))
        if response.ok :
            json_response = response.json()
            if petId == json_response['id']:
                return json_response
        return json.dumps({'error': 'Pet id not found.'})
    except requests.exceptions.HTTPError as e:
        if response.status_code == 404:
            return json.dumps({'error': 'Pet id not found.'})
        else:
            return json.dumps({'error': 'Error with API.'})

def retrieve_user_info(username: str) -> str:
    try:
        url = 'https://petstore3.swagger.io/api/v3/user/' + username
        response = requests.get(url)
        # Raise an exception if the response was unsuccessful
        response.raise_for_status()
        if response.ok :
            json_response = response.json()
            if username == json_response['username']:
                return json_response
        return json.dumps({'error': 'Username id not found.'})
    except requests.exceptions.HTTPError as e:
        if response.status_code == 404:
            return json.dumps({'error': 'Username not found.'})
        else:
            return json.dumps({'error': 'Error with API.'})

- In order for Mistral models to understand the functions, we need to outline the function specifications with a JSON schema. Specifically, we need to describe the type, function name, function description, function parameters, and the required parameter for the function. Since we have two functions here, let’s list two function specifications in a list.

In [2]:
from mistral_common.protocol.instruct.tool_calls import Function, Tool

user_tools=[
        Tool(
            function=Function(
                name="retrieve_pet_info",
                description="Find pet by ID",
                parameters={
                    "type": "object",
                    "properties": {
                        "petId": {
                        "type": "integer",
                        "description": "The pet id.",
                        }
                    },
                    "required": ["petId"],
                },
            )
        ),
        Tool(
            function=Function(
                name="retrieve_user_info",
                description="Get user by user name",
                parameters={
                    "type": "object",
                    "properties": {
                        "username": {
                        "type": "string",
                        "description": "The username.",
                        }
                    },
                    "required": ["username"],
                },
            )
        )
    ]



- Then we organize the two functions into a dictionary where keys represent the function name, and values are the function with the df defined. This allows us to call each function based on its function name.

In [3]:
import functools

names_to_functions = {
  'retrieve_pet_info': functools.partial(retrieve_pet_info, petId=''),
  'retrieve_user_info': functools.partial(retrieve_user_info, username='')  
}

### User query

Suppose a user asks the following question: “What’s the status of my Pet 1?” A standalone LLM would not be able to answer this question, as it needs to query the business logic backend to access the necessary data. But what if we have an exact tool we can use to answer this question? We could potentially provide an answer!

In [91]:
from mistral_common.protocol.instruct.messages import UserMessage

user_messages=[UserMessage(content="What's the status of my Pet 1?")
        ]

## Step 2. Model: Generate function arguments 

How do Mistral models know about these functions and know which function to use? We provide both the user query and the tools specifications to Mistral models. The goal in this step is not for the Mistral model to run the function directly. It’s to 1) determine the appropriate function to use , 2) identify if there is any essential information missing for a function, and 3) generate necessary arguments for the chosen function.

In [89]:
from mistral_inference.model import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

from mistral_common.protocol.instruct.request import ChatCompletionRequest

- We specify the location to load the mistral model and tokenizer.  These models are downloaded from [mistral-inference](https://github.com/mistralai/mistral-inference) repo. We have downloaded the mistral 7BInstructv3 model

In [6]:
tokenizer = MistralTokenizer.from_file("./mistral_models/7B_instruct/tokenizer.model.v3")  # change to extracted tok>
model = Transformer.from_folder("./mistral_models/7B_instruct")  # change to extracted model dir

In [92]:
completion_request = ChatCompletionRequest(tools=user_tools, messages=user_messages,)

In [8]:
tokens = tokenizer.encode_chat_completion(completion_request).tokens


max_tokens control the output of model as tokens, set it based on required function call.

In [9]:
out_tokens, _ = generate([tokens], model, max_tokens=25, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)

In [10]:
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])
#remove any blank space from token output
result = result.strip()
result

'[{"name": "retrieve_pet_info", "arguments": {"petId": 1}}]'

## Step 3. User: Execute function to obtain tool results

How do we execute the function? Currently, it is the user’s responsibility to execute these functions and the function execution lies on the user side. In the future, we may introduce some helpful functions that can be executed server-side.

Let’s extract some useful function information from model response including function_name and function_params. It’s clear here that our Mistral model has chosen to use the function `retrieve_payment_status` with the parameter `transaction_id` set to T1001.

In [11]:
import json
tool_call = json.loads(result)
function_name = tool_call[0]["name"]
function_params = (tool_call[0]["arguments"]) 
print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)


function_name:  retrieve_pet_info 
function_params:  {'petId': 1}


In [12]:
function_result = names_to_functions[function_name](**function_params)
function_result

{'id': 1,
 'category': {'id': 1, 'name': 'Bears'},
 'name': 'Zoe',
 'photoUrls': ['string'],
 'tags': [{'id': 1, 'name': 'string'}],
 'status': 'available'}

TODO - parse open api spec for dynamic tool definition creation

In [ ]:
!pip install --upgrade  prance openapi-spec-validator


In [1]:
import prance

def load_openapi_spec(file_path):
    """
    Loads an OpenAPI specification from a JSON file.
    
    Args:
        file_path (str): The path to the OpenAPI specification file.
        
    Returns:
        The parsed OpenAPI specification object.
    """
    # Parse the OpenAPI specification file
    parser = prance.ResolvingParser(file_path, backend='openapi-spec-validator')

    #parser = prance.ResolvingParser(file_path)
    spec = parser.specification

    return spec


In [2]:
# Load the OpenAPI specification
spec = load_openapi_spec('openapi.json')
print(f"Paths: {list(spec['paths'].keys())}")
# Display json for get
#spec['paths']['/pet/{petId}']['get']
function_name=spec['paths']['/pet/{petId}']['get']['operationId']
print(function_name)
function_description=spec['paths']['/pet/{petId}']['get']['description']
print(function_description)
function_parameters=spec['paths']['/pet/{petId}']['get']['parameters']
print(function_parameters)

Paths: ['/pet', '/pet/findByStatus', '/pet/findByTags', '/pet/{petId}', '/pet/{petId}/uploadImage', '/store/inventory', '/store/order', '/store/order/{orderId}', '/user', '/user/createWithList', '/user/login', '/user/logout', '/user/{username}']
getPetById
Returns a single pet
[{'name': 'petId', 'in': 'path', 'description': 'ID of pet to return', 'required': True, 'schema': {'type': 'integer', 'format': 'int64'}}]


In [60]:
# Load the OpenAPI specification
spec = load_openapi_spec('openapi.json')
print(f"Paths: {list(spec['paths'].keys())}")
# Display json for get
#spec['paths']['/pet/{petId}']['get']
function_name=spec['paths']['/user/{username}']['get']['operationId']
print(function_name)
function_description=spec['paths']['/user/{username}']['get']['description']
print(function_description)
function_parameters=spec['paths']['/user/{username}']['get']['parameters']
print(function_parameters)

Paths: ['/pet', '/pet/findByStatus', '/pet/findByTags', '/pet/{petId}', '/pet/{petId}/uploadImage', '/store/inventory', '/store/order', '/store/order/{orderId}', '/user', '/user/createWithList', '/user/login', '/user/logout', '/user/{username}']
getUserByName

[{'name': 'username', 'in': 'path', 'description': 'The name that needs to be fetched. Use user1 for testing. ', 'required': True, 'schema': {'type': 'string'}}]


In [3]:
from mistral_common.protocol.instruct.tool_calls import Function, Tool

In [4]:
toolFunct = Function

In [52]:
func_parameters = {
    "type": "object",
    "properties": {
        function_parameters[0]['name']: {
            "type": function_parameters[0]['schema']['type'],
            "description": function_parameters[0]['description']
        }
    },
    "required": [function_parameters[0]['name']]
}

In [53]:
import jsonschema

# Define the input data
input_data = {"petId": 123}

# Validate the input data against the schema
jsonschema.validate(instance=input_data, schema=parameters)

In [64]:
spec = load_openapi_spec('openapi.json')
function_name=spec['paths']['/pet/{petId}']['get']['operationId']
function_description=spec['paths']['/pet/{petId}']['get']['description']
function_parameters=spec['paths']['/pet/{petId}']['get']['parameters']
func_parameters = {
    "type": "object",
    "properties": {
        function_parameters[0]['name']: {
            "type": function_parameters[0]['schema']['type'],
            "description": function_parameters[0]['description']
        }
    },
    "required": [function_parameters[0]['name']]
}

user_function_1= Function(name = function_name, description = function_description, parameters = func_parameters, )

In [65]:
spec = load_openapi_spec('openapi.json')
function_name=spec['paths']['/user/{username}']['get']['operationId']
function_description=spec['paths']['/user/{username}']['get']['description']
function_parameters=spec['paths']['/user/{username}']['get']['parameters']
func_parameters = {
    "type": "object",
    "properties": {
        function_parameters[0]['name']: {
            "type": function_parameters[0]['schema']['type'],
            "description": function_parameters[0]['description']
        }
    },
    "required": [function_parameters[0]['name']]
}

user_function_2= Function(name = function_name, description = function_description, parameters = func_parameters, )

In [66]:
user_tool_1 = Tool(function = user_function_1)

In [67]:
user_tool_2 = Tool(function = user_function_2)

In [69]:
user_tools = [user_tool_1, user_tool_2]

In [47]:
print(toolFunct.name + ' - ' + toolFunct.description +  ' - ' + str(toolFunct.parameters)) 

getPetById - Returns a single pet - {'type': 'object', 'properties': {'petId': {'type': 'integer', 'description': 'ID of pet to return'}}, 'required': ['petId']}


In [84]:
objs = [['pet','petId'], ['user', 'username'], ['store/order','orderId']]
params = ['operationId', 'description',  'parameters']
user_tools = []

In [88]:
for obj in objs:
    resource, field = obj
    path = '/' + resource + '/{' + field + '}'
    function_name=spec['paths'][path]['get'][params[0]]
    function_description=spec['paths'][path]['get'][params[1]]
    function_parameters=spec['paths'][path]['get'][params[2]]
    func_parameters = {
    "type": "object",
    "properties": {
        function_parameters[0]['name']: {
            "type": function_parameters[0]['schema']['type'],
            "description": function_parameters[0]['description']
        }
    },
    "required": [function_parameters[0]['name']]
}
    user_function= Function(name = function_name, description = function_description, parameters = func_parameters, )
    user_tool = Tool(function = user_function)
    user_tools.append(user_tool)
    print(f"Resource: {resource}, Field: {field}, Path : {path}")

Resource: pet, Field: petId, Path : /pet/{petId}
Resource: user, Field: username, Path : /user/{username}
Resource: store/order, Field: orderId, Path : /store/order/{orderId}


In [93]:
completion_request = ChatCompletionRequest(tools=user_tools, messages=user_messages,)

In [79]:
spec = load_openapi_spec('openapi.json')
path = '/' + obj_1_name + '/{' + obj_1_id + '}'
params = ['operationId', 'description',  'parameters']
function_name=spec['paths'][path]['get'][params[0]]
function_description=spec['paths'][path]['get'][params[1]]
function_parameters=spec['paths'][path]['get'][params[2]]
func_parameters = {
    "type": "object",
    "properties": {
        function_parameters[0]['name']: {
            "type": function_parameters[0]['schema']['type'],
            "description": function_parameters[0]['description']
        }
    },
    "required": [function_parameters[0]['name']]
}

user_function_1= Function(name = function_name, description = function_description, parameters = func_parameters, )

In [74]:
path = '/' + obj_1_name + '/{' + obj_1_id + '}'
function_name=spec['paths'][path]['get']['operationId']
function_name

'getPetById'

In [11]:
# Print the attributes and their values using vars()
print("\nAttributes and values using vars():")
for attribute, value in vars(toolFunct).items():
    print(f"{attribute}: {value}")


Attributes and values using vars():
__module__: mistral_common.protocol.instruct.tool_calls
__annotations__: {'name': <class 'str'>, 'description': <class 'str'>, 'parameters': typing.Dict[str, typing.Any]}
model_config: {'extra': 'forbid', 'validate_default': True, 'use_enum_values': True}
__class_vars__: set()
__private_attributes__: {}
__doc__: None
__abstractmethods__: frozenset()
_abc_impl: <_abc._abc_data object at 0x7e740709f500>
__pydantic_custom_init__: False
__pydantic_post_init__: None
__pydantic_decorators__: DecoratorInfos(validators={}, field_validators={}, root_validators={}, field_serializers={}, model_serializers={}, model_validators={}, computed_fields={})
__pydantic_generic_metadata__: {'origin': None, 'args': (), 'parameters': ()}
__pydantic_complete__: True
__pydantic_parent_namespace__: {'__name__': 'mistral_common.protocol.instruct.tool_calls', '__doc__': <pydantic._internal._model_construction._PydanticWeakRef object at 0x7e7420d6b0d0>, '__package__': 'mistral_

In [13]:
print("Attributes and values using dir():")
for attribute in dir(toolFunct):
    if not attribute.startswith("__"):
        print(f"{attribute}: {getattr(toolFunct, attribute)}")

Attributes and values using dir():
_abc_impl: <_abc._abc_data object at 0x7e740709f500>
_calculate_keys: <function BaseModel._calculate_keys at 0x7e74070720e0>
_check_frozen: <function BaseModel._check_frozen at 0x7e74070712d0>
_copy_and_set_values: <function BaseModel._copy_and_set_values at 0x7e7407071fc0>
_get_value: <bound method BaseModel._get_value of <class 'mistral_common.protocol.instruct.tool_calls.Function'>>
_iter: <function BaseModel._iter at 0x7e7407071f30>
construct: <bound method BaseModel.construct of <class 'mistral_common.protocol.instruct.tool_calls.Function'>>
copy: <function BaseModel.copy at 0x7e7407071c60>
description: Returns a single pet
dict: <function BaseModel.dict at 0x7e7407071870>
from_orm: <bound method BaseModel.from_orm of <class 'mistral_common.protocol.instruct.tool_calls.Function'>>
json: <function BaseModel.json at 0x7e7407071900>
model_computed_fields: {}
model_config: {'extra': 'forbid', 'validate_default': True, 'use_enum_values': True}
model_c